In [4]:
import requests
import time

api_key = 'ec3f3684d92da1077a58e0270bd86429'
base_url = 'https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=1'
base_url_genre='https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US'

# Function to fetch movie details
def get_movie_details(page_num):
    endpoint = f'{base_url}/discover/movie'
    params = {
        'api_key': api_key,
        'page': page_num
    }
    response = requests.get(endpoint, params=params)
    return response.json()['results']

# Fetch data from multiple pages
all_movies = []
for page_num in range(1, 472):  # 471 pages of movies
    movies = get_movie_details(page_num)
    all_movies.extend(movies)
    time.sleep(0.2)  # To avoid hitting API rate limits

# Explore the data structure returned by the API
print(all_movies[0])  # Check the structure of a single movie entry

{'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'genre_ids': [18, 80], 'id': 278, 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 142.177, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'release_date': '1994-09-23', 'title': 'The Shawshank Redemption', 'video': False, 'vote_average': 8.71, 'vote_count': 25281}


In [5]:
# Function to fetch genre names
def get_genre_names():
    endpoint = f'{base_url_genre}/genre/movie/list'
    params = {'api_key': api_key}
    response = requests.get(endpoint, params=params)
    return response.json()['genres']

genre_names = get_genre_names()
print(genre_names)  # Check the structure of genre data


[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]


In [6]:
# Function to map genre IDs to names
def map_genre_ids(genre_ids):
    return [genre['name'] for genre in genre_names if genre['id'] in genre_ids]

# Create a list to hold organized movie data
movies_data = []

# Iterate through each movie to collect required information
for movie in all_movies:
    movie_name = movie['title']
    movie_description = movie['overview']
    genre_ids = movie['genre_ids']
    movie_genres = map_genre_ids(genre_ids)

    # Append the collected data as a dictionary
    movies_data.append({
        'Name': movie_name,
        'Description': movie_description,
        'Genres': movie_genres
    })

# Explore the structure of the movies_data list
print(movies_data[0])  # Check the structure of a single movie entry


{'Name': 'The Shawshank Redemption', 'Description': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'Genres': ['Crime', 'Drama']}


In [9]:
import csv
import json

# Save as CSV
with open('movies_dataset.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Name', 'Description', 'Genres']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for movie in movies_data:
        writer.writerow(movie)


with open('movies_dataset.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(movies_data, jsonfile, ensure_ascii=False, indent=4)



In [11]:
import csv

# Assume movies_data is your list of movie data (similar to your provided format)

# Convert list of genres to comma-separated strings
for movie in movies_data:
    movie['Genres'] = ', '.join(movie['Genres'])

# Save as CSV
with open('movies_dataset.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Name', 'Description', 'Genres']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for movie in movies_data:
        writer.writerow(movie)


In [14]:
df = pd.read_csv('movies_dataset.csv')


In [15]:
df.head()

,Name,Description,Genres
0,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Crime, Drama"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Crime, Drama"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"Crime, Drama"
3,Schindler's List,The true story of how businessman Oskar Schind...,"Drama, History, War"
4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy, Drama, Romance"


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

In [25]:
df.shape

(9072, 3)

In [26]:
df['Description'][3]

'The true story of how businessman Oskar Schindler saved over a thousand Jewish lives from the Nazis while they worked as slaves in his factory during World War II.'

In [28]:
def lower_text(text):
  text=text.lower()
  return text

In [30]:
def remove_html(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [34]:
import string

In [35]:
exclude=string.punctuation

In [36]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [37]:
def remove_punct(text):
  return text.translate(str.maketrans('','',exclude))

In [39]:
from textblob import TextBlob

In [41]:
def spelling(text):
  txtblb=TextBlob(text)
  return txtblb.correct().string


In [48]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [49]:
def remove_stopwords(text):
  new_text=[]

  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')

    else:
      new_text.append(word)

  x=new_text[:]
  new_text.clear()
  return " ".join(x)

In [50]:
remove_stopwords(df['Description'][55])

'With dreams  diving abroad, Tsuneo gets  job assisting Josee,  artist whose imagination takes  far beyond  wheelchair. But   tide turns  them,  push    places  never thought possible,  inspire  love fit   storybook.'

In [51]:
from nltk.stem.porter import PorterStemmer

In [52]:
ps=PorterStemmer()

In [53]:
def stemming(text):
  return " ".join([ps.stem(word) for word in text.split()])

In [55]:
def text_prep(text):
  lower_text(text)
  remove_html(text)
  remove_punct(text)
  spelling(text)
  remove_stopwords(text)
  stemming(text)

  return text


In [ ]:
df['Description'].apply(text_prep)